In [1]:
CUDA_INDEX = 0
NAME = 'PubMed'
CLASSES = 3

In [2]:
import sys
sys.path.insert(0, '../..')
sys.path.insert(0, '../../pyged/lib')

In [25]:
import os
import pickle
import random
import time

import IPython as ipy
import matplotlib.pyplot as plt
import numpy as np
import torch
torch.cuda.set_device(CUDA_INDEX)
torch.backends.cudnn.benchmark = True
import torch.optim
import torch_geometric as tg
import torch_geometric.data
from tqdm.auto import tqdm

from neuro import config, datasets, index, metrics, models, train, utils, viz
import pyged

from importlib import reload
reload(config)
reload(datasets)
reload(index)
reload(metrics)
reload(models)
reload(pyged)
reload(train)
reload(utils)
reload(viz)

<module 'neuro.viz' from '../../neuro/viz.py'>

In [4]:
graphs = utils.remove_extra_attrs(utils.label_graphs(tg.datasets.CitationFull(root=f'../data/{NAME}/tg', name=f'{NAME}')))

move labels to node attrs


graphs:   0%|          | 0/1 [00:00<?, ?it/s]

remove extra attrs


graphs:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# tic = time.time()
# nbrs = datasets.decompose(graphs, n_hops=2)
# toc = time.time()
# torch.save(nbrs, f'../data/{NAME}/nbrs.pt')
# tqdm.write(f'neighborhood decomposition time: {toc-tic:.3} s')
nbrs = torch.load(f'../data/{NAME}/nbrs.pt')

In [9]:
queries = datasets.make_queries(nbrs, n_queries=10, n_hops=2, trav_prob=0.4, node_lim=25)

sample queries from targets


sampled queries:   0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
model = models.NormSEDModel(8, CLASSES, 64, 64)
model.load_state_dict(torch.load(f'../runlogs/{NAME}/1621887700.5921354/best_model.pt', map_location='cpu'))
model = model.eval()
model = model.to(config.device)

In [11]:
tic = time.time()
with torch.no_grad():
    target_embs = model.embed_model(tg.data.Batch.from_data_list(nbrs).to(config.device))
toc = time.time()
tqdm.write(f'total target embedding time: {toc-tic:.3} s')
tqdm.write(f'per target embedding time: {(toc-tic)/len(nbrs):.3} s')

total target embedding time: 1.75 s
per target embedding time: 8.87e-05 s


In [12]:
tic = time.time()
with torch.no_grad():
    query_embs = model.embed_model(tg.data.Batch.from_data_list(queries).to(config.device))
toc = time.time()
tqdm.write(f'total query embedding time: {toc-tic:.3} s')
tqdm.write(f'per query embedding time: {(toc-tic)/len(queries):.3} s')

total query embedding time: 0.00816 s
per query embedding time: 0.000816 s


In [72]:
temb = target_embs.repeat(100,1)
qemb = query_embs

# Linear Scan

## Unvectorised, CPU, threads=1

In [38]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.LinearScan(temb, dist_fn=utils.norm_sed_func)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

slow (pythonic) linear scan index
config.device: cpu
preprocessing time: 0.0179 s


In [39]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.658	0.652	0.609	0.62	0.61	0.632	


In [40]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.71	0.682	0.673	0.721	0.78	0.886	0.981	


## Vectorised, CPU, threads=1

In [73]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.FastLinearScan(temb, dist_fn=utils.norm_sed_func)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

fast (vectorised) linear scan index
config.device: cpu
preprocessing time: 0.421 s


In [74]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
1.37 / 1.4 / 1.47 / 1.44 / 1.5 / 1.46 / 


In [75]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
1.39 / 1.42 / 1.51 / 1.46 / 1.48 / 1.49 / 1.42 / 


## Vectorised, GPU

In [44]:
tic = time.time()
config.device = torch.device('cuda')
index_str = index.FastLinearScan(temb, dist_fn=utils.norm_sed_func)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

fast (vectorised) linear scan index
config.device: cuda
preprocessing time: 0.0155 s


In [45]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.00384	0.00355	0.00374	0.00345	0.00365	0.00317	


In [46]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.00297	0.0029	0.00291	0.00312	0.00403	0.00273	0.00274	


# Quasi Metric Tree

## Unvectorised, CPU, threads=1, max_leaf_size=8

In [47]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=8)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cpu


  0%|          | 0/19717 [00:00<?, ?it/s]

distance computations: 259532
internal nodes: 1925 / 7701
leaf nodes: 5776 / 7701
big leaf nodes: 30 / 5776
max leaf size: 58
preprocessing time: 0.963 s


In [48]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=True, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.511	0.499	0.542	0.552	0.641	0.551	


In [49]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=True, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.516	0.532	0.524	0.58	0.611	0.783	0.993	


## Unvectorised, CPU, threads=1, max_leaf_size=64

In [50]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=64)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cpu


  0%|          | 0/19717 [00:00<?, ?it/s]

distance computations: 193296
internal nodes: 246 / 985
leaf nodes: 739 / 985
big leaf nodes: 0 / 739
max leaf size: 64
preprocessing time: 0.277 s


In [51]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=True, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.453	0.476	0.476	0.547	0.561	0.577	


In [52]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=True, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.494	0.433	0.462	0.522	0.602	0.689	0.869	


## Vectorised, CPU, threads=1, max_leaf_size=8

In [53]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=8)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cpu


  0%|          | 0/19717 [00:00<?, ?it/s]

distance computations: 259532
internal nodes: 1925 / 7701
leaf nodes: 5776 / 7701
big leaf nodes: 30 / 5776
max leaf size: 58
preprocessing time: 0.838 s


In [54]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.31	0.37	0.354	0.361	0.392	0.355	


In [55]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.4	0.414	0.463	0.505	0.521	0.644	0.806	


## Vectorised, CPU, threads=1, max_leaf_size=64

In [76]:
tic = time.time()
config.device = torch.device('cpu')
torch.set_num_threads(1)
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=64)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cpu


  0%|          | 0/1971700 [00:00<?, ?it/s]

distance computations: 35936600
internal nodes: 11740 / 46961
leaf nodes: 35221 / 46961
big leaf nodes: 18809 / 35221
max leaf size: 5800
preprocessing time: 35.6 s


In [77]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
1.81 / 2.04 / 1.94 / 2.09 / 2.17 / 1.89 / 


In [78]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
7.76 / 7.88 / 7.79 / 7.86 / 8.07 / 8.57 / 9.5 / 


## Vectorised, GPU, max_leaf_size=8

In [59]:
tic = time.time()
config.device = torch.device('cuda')
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=8)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cuda


  0%|          | 0/19717 [00:00<?, ?it/s]

distance computations: 259732
internal nodes: 1940 / 7761
leaf nodes: 5821 / 7761
big leaf nodes: 29 / 5821
max leaf size: 35
preprocessing time: 1.79 s


In [60]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.906	0.974	0.968	1.1	1.14	1.07	


In [61]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
1.21	1.16	1.28	1.49	1.82	2.48	3.33	


## Vectorised, GPU, max_leaf_size=64

In [62]:
tic = time.time()
config.device = torch.device('cuda')
index_str = index.QuasiMetricTree(temb, dist_fn=utils.norm_sed_func, max_leaf_size=64)
toc = time.time()
tqdm.write(f'preprocessing time: {toc-tic:.3} s')

construct quasi metric tree for asymmetric/symmetric distance function
config.device: cuda


  0%|          | 0/19717 [00:00<?, ?it/s]

distance computations: 193304
internal nodes: 246 / 985
leaf nodes: 739 / 985
big leaf nodes: 0 / 739
max leaf size: 64
preprocessing time: 0.252 s


In [63]:
out = ''
for r in [0,1,2,4,8,12]:
    tic = time.time()
    for q in tqdm(qemb, f'r = {r} | queries'):
        index_str.range_query(q, r, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

r = 0 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 2 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 4 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 8 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

r = 12 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.156	0.145	0.151	0.164	0.209	0.185	


In [64]:
out = ''
for k in [1,10,20,100,200,500,1000]:
    tic = time.time()
    for q in tqdm(qemb, f'k = {k} | queries'):
        index_str.knn_query(q, k, slow=False, verbose=False)
    toc = time.time()
    out += f'{(toc-tic)/qemb.shape[0]:.3} / '
tqdm.write('')
tqdm.write('time in s:')
tqdm.write(out)

k = 1 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 10 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 20 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 100 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 200 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 500 | queries:   0%|          | 0/10 [00:00<?, ?it/s]

k = 1000 | queries:   0%|          | 0/10 [00:00<?, ?it/s]


time in s:
0.646	0.688	0.719	0.972	1.21	1.83	2.58	
